In [1]:
# 搭建一个12导联训练框架
# resnet18中留出蒸馏接口（即输出最后一层的特征） （distill=True时多输出一个特征即可）
# 训练流程中，留出模型保存接口（直接保存整个模型，以便后续学生学习教师时候，教师直接调用）
# 留出结果保存接口，或者输出也行，一个个手算。
# 数据的读取需要补充一些程序
# 目前distill只支持resnet18的basic和DGGR（GRL）
# 2分类训练框架加入GRL，同时加入distill=True
# 惩罚项增加了一部分，因此学习率应当适当减小？
有个问题，怎么感觉加了GRL学习后，loss这么大了？

SyntaxError: invalid character '，' (U+FF0C) (3297128182.py, line 9)

In [3]:
!gpustat

vclab-gpuserver-57          Fri Mar  8 12:47:48 2024  515.65.01
[0] NVIDIA GeForce RTX 3090 | 29'C,   0 % |  1124 / 24576 MB | yuzhenqin(813M)
[1] NVIDIA GeForce RTX 3090 | 30'C,   0 % |   311 / 24576 MB |
[2] NVIDIA GeForce RTX 3090 | 29'C,   0 % |  2270 / 24576 MB | yuzhenqin(1959M)
[3] NVIDIA GeForce RTX 3090 | 29'C,   0 % |  4180 / 24576 MB | yuzhenqin(3869M)
[4] NVIDIA GeForce RTX 3090 | 31'C,   0 % |   311 / 24576 MB |
[5] NVIDIA GeForce RTX 3090 | 29'C,   0 % |   311 / 24576 MB |
[6] NVIDIA GeForce RTX 3090 | 56'C,  21 % | 18736 / 24576 MB | robotics(18425M)
[7] NVIDIA GeForce RTX 3090 | 61'C,  20 % | 18736 / 24576 MB | robotics(18425M)


In [4]:
# box1

# 库函数的调用
from models.DG_models import generate_model
from datasets import train_loader,val_loader,test_loader
import pandas as pd
import numpy as np
import ast
import matplotlib.pyplot as plt
import os
os.environ["CUDA_VISIBLE_DEVICES"] ='5'
device='cuda:0'
import wfdb
from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer
from scipy import signal
from tqdm import tqdm
import math
import time
import torch
from torch import nn
from torchsummary import summary
import sklearn.metrics
import csv
from utils import *
import copy

warm_epoch=2
total_epoch=20
base_lr=2e-3
batch_size=128
max_lr=base_lr*batch_size/256
min_lr=max_lr/100
numleads=12
distill=True
# domain generalization
DG_method='teacher_self_distill_leadI_GRL'
# None #'DG_GR' #'MMD' # 'CORAL'  # 'IRM' # 'CausIRL_MMD' # 'CausIRL_CORAL' # 'cls_awr_ali' # 'DG_GR+IRM' 
# 'DG_GR_ensemble' # 'origin_add_fft_amp' # 'origin_add_fft_phase' # 'origin_add_fft_amp_phase' # 'origin_add_fft_amp_DGGR'
# 'DGGR_smooth' #'origin_add_fft_amp_DGGR_smooth'
add_recons=None #'fft','ksqi','psqi','ssqi','bass',None
#ensemble_recons=['fft','ksqi'] # ['fft','ksqi','psqi','ssqi','bass'] 
MMD_iter=4
MMD_gamma=1.
irm_lambda=1e2
GRL_loss_beta=0.05
irm_penalty_anneal_iters=10000000 # don't penalty #500
DGGR_smooth_eps=0.3
ali_weight=0.5
train_val_test_split=[0.8,0.2,1] # 1 for inter
smote=False
SAM_flag=False
base_optimizer=torch.optim.Adam
loss_fn=nn.CrossEntropyLoss()
# cnn_Ag
# xresnet18,xresnet34,xresnet50,xresnet101,xresnet152
# inception1d
# resnet18,resnet34,resnet50,resnet101,resnet152
# resnet1d_wang
# seresnet_big_18,seresnet_big_34,seresnet_big_50,seresnet_big_101,seresnet_big_152
# densenet1d

base_model='cnn_Ag_teacher_self_distill_leadI'
exp_num=5
class_num=3

if DG_method in ['DGGR_smooth','origin_add_fft_amp_DGGR_smooth'] :
    param_flag='AF_2class_smote-'+str(smote)+'-SAM-'+str(SAM_flag)+'-DG_method-'+str(DG_method)+'-beta-'+str(int(GRL_loss_beta*100))+'-eps-'+str(int(DGGR_smooth_eps*100))
else:
    param_flag='AF_2class_smote-'+str(smote)+'-SAM-'+str(SAM_flag)+'-DG_method-'+str(DG_method)+'-beta-'+str(int(GRL_loss_beta*100))

print('ok')

# 10次实验取平均

auc_dict_list=[]
pre_dict_list=[]
rec_dict_list=[]
f1_dict_list=[]
f1_normal_dict_list=[]
f1_AF_dict_list=[]
f1_other_dict_list=[]
acc_dict_list=[]

start_t=time.time()

train_data_list=['C_12','A_12','L_12','N_12','B_12','H_12'] # T,A,L,N,H,B       
val_data_list_A=['A_12']
val_data_list_B=['B_12']
val_data_list_C=['C_12']
val_data_list_H=['H_12']
val_data_list_L=['L_12']
val_data_list_N=['N_12']

#     test_T_data_list=['T_3cls']

train_datasloader_instance=train_loader(dataname=train_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,smote=smote,maple=True if DG_method=='maple' else False,class_num=class_num)
train_datasloader=train_datasloader_instance.loader()
val_datasloader_A=val_loader(dataname=val_data_list_A,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_B=val_loader(dataname=val_data_list_B,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_C=val_loader(dataname=val_data_list_C,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_H=val_loader(dataname=val_data_list_H,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_L=val_loader(dataname=val_data_list_L,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
val_datasloader_N=val_loader(dataname=val_data_list_N,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()
#     test_datasloader=test_loader(dataname=test_data_list,train_val_test_split=train_val_test_split,bs=batch_size,num_workers=2,class_num=class_num).loader()


for exp_idx in range(exp_num):
    
    print('current_exp_idx=',exp_idx)
    
    recons_model=None

    model=generate_model(base_model=base_model,input_channels=numleads, num_classes=class_num,DG_method=DG_method,domain_classes=len(train_data_list),distill=distill).to(device)
    optimizer=generate_optimizer(base_optimizer=base_optimizer,model_parameters=model.parameters(),SAM_flag=SAM_flag,weight_decay=1e-4)
    best_val_f1 = 0.0
    best_model_weights = None

    mdst_path='/home/yangli/AF_DG_OODG/save_model_states'
    mdst_name=str(int(time.time()))+'_ABCLNH_TR_proposed_cnnAg_zong_f1.pth'

    for epoch in range(total_epoch):
        print('_____________________________________')
        print(' ')
        time.sleep(0.1)
        train_one_epoch(epoch, warm_epoch, total_epoch, model,max_lr,min_lr, loss_fn, optimizer, \
                        base_optimizer, train_datasloader, device, \
                         DG_method=DG_method,MMD_iter=MMD_iter,MMD_gamma=MMD_gamma, \
                         irm_lambda=irm_lambda,irm_penalty_anneal_iters=irm_penalty_anneal_iters, \
                        ali_weight=ali_weight,SAM_operate=SAM_flag,recons_model=recons_model,add_recons=add_recons,  \
                       DGGR_smooth_eps=DGGR_smooth_eps,theta=0,GRL_loss_beta=GRL_loss_beta,class_num=class_num,distill=distill, \
                        domain_class=len(train_data_list))

        time.sleep(0.1)
        _,_,_,f1_val_A,_,_,_,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_A, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num,distill=distill) 
        print('f1_val_A=0.'+str(f1_val_A))
        _,_,_,f1_val_B,_,_,_,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_B, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num,distill=distill) 
        print('f1_val_B=0.'+str(f1_val_B))
        _,_,_,f1_val_C,_,_,_,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_C, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num,distill=distill) 
        print('f1_val_C=0.'+str(f1_val_C))
        _,_,_,f1_val_H,_,_,_,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_H, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num,distill=distill) 
        print('f1_val_H=0.'+str(f1_val_H))
        _,_,_,f1_val_L,_,_,_,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_L, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num,distill=distill) 
        print('f1_val_L=0.'+str(f1_val_L))
        _,_,_,f1_val_N,_,_,_,_=val_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, val_datasloader_N, device,DG_method=DG_method,class_F1=True,recons_model=recons_model,add_recons=add_recons,class_num=class_num,distill=distill) 
        print('f1_val_N=0.'+str(f1_val_N))
        
        f1_val=int(f1_val_A)+int(f1_val_B)+int(f1_val_C)+int(f1_val_H)+int(f1_val_L)+int(f1_val_N)

        if int(f1_val) > best_val_f1:
            best_val_f1 = int(f1_val)
            best_model_weights = model.state_dict()  # Save the current model weights
            

            # 保存模型参数到指定文件夹和文件名  


            torch.save(best_model_weights, os.path.join(mdst_path, mdst_name))
            best_epoch=epoch
            
    print(best_epoch)

#     # Load the best model weights before testing
#     model.load_state_dict(best_model_weights)

#     auc_,pre_,rec_,f1_,acc_=  \
#         test_one_epoch(epoch,warm_epoch, total_epoch, model, loss_fn, test_datasloader, device,DG_method=DG_method,class_F1=False,recons_model=recons_model,add_recons=add_recons,class_num=class_num,distill=distill) 
        
#     auc_dict_list.append(int(auc_)  )    
#     pre_dict_list.append(int(pre_)  )
#     rec_dict_list.append(int(rec_)  )
#     f1_dict_list.append(int(f1_)  )
#     acc_dict_list.append(int(acc_)  )

# auc_dict=str(np.mean(auc_dict_list))
# pre_dict=str(np.mean(pre_dict_list))
# rec_dict=str(np.mean(rec_dict_list))
# f1_dict=str(np.mean(f1_dict_list))
# acc_dict=str(np.mean(acc_dict_list))

# # 写入csv

# header=[
#         'train\\test\t',
#         'auc','pre','rec','f1','acc\t',]
# #         'auc','pre','rec','f1','normal-f1','AF-f1','other-f1','acc\t',]

#         # round

# results=[
#         [
#         'T',
#         auc_dict,pre_dict,rec_dict,f1_dict,acc_dict+'\t',
#          ],
#         ]

# with open('log/AF_2cls_distill_teacher_self_distill_GRL_dom_ABCLN_T_single_lead_simple_conv_net_multione_0111_box1.csv','w',encoding='utf-8')as file_obj:
#     writer=csv.writer(file_obj)
#     writer.writerow(header)
#     for result in results:
#         writer.writerow(result)

ok
train data C_12
train data size : 24194
train label class :  [ 6719.  2039. 15436. 24194.     0.     0.     0.     0.     0.]
train data A_12
train data size : 8274
train label class :  [1393.  480. 6401.    0. 8274.    0.    0.    0.    0.]
train data L_12
train data size : 16824
train label class :  [13511.  1158.  2155.     0.     0. 16824.     0.     0.     0.]
train data N_12
train data size : 8516
train label class :  [1462. 1769. 5285.    0.    0.    0. 8516.    0.    0.]
train data B_12
train data size : 5501
train label class :  [ 742.  859. 3900.    0.    0.    0.    0. 5501.    0.]
train data H_12
train data size : 20616
train label class :  [11152.   557.  8907.     0.     0.     0.     0.     0. 20616.]
total train data shape =  (83925, 1000, 12)
total train label shape =  (83925, 9)
total train data class =  [34979.  6862. 42084. 24194.  8274. 16824.  8516.  5501. 20616.]
val data A_12
val data size : 2069
val label class :  [ 359.   90. 1620.]
total val data shape =  

epoch= 10
train_running_loss= 0.6658033693348293
epoch= 10
val_loss= 0.4797624595025006
________________________________
f1_val_A=0.742
epoch= 10
val_loss= 0.35870814052495087
________________________________
f1_val_B=0.826
epoch= 10
val_loss= 0.20053446479141712
________________________________
f1_val_C=0.918
epoch= 10
val_loss= 0.3272719128829677
________________________________
f1_val_H=0.860
epoch= 10
val_loss= 0.3099690707343997
________________________________
f1_val_L=0.839
epoch= 10
val_loss= 0.2703058956300511
________________________________
f1_val_N=0.897
_____________________________________
 
epoch= 11
train_running_loss= 0.6821408713557114
epoch= 11
val_loss= 0.46649560858221617
________________________________
f1_val_A=0.766
epoch= 11
val_loss= 0.3860050954601981
________________________________
f1_val_B=0.802
epoch= 11
val_loss= 0.21682757511734962
________________________________
f1_val_C=0.916
epoch= 11
val_loss= 0.30824634796235617
________________________________
f1

epoch= 3
val_loss= 0.24229365773499012
________________________________
f1_val_C=0.906
epoch= 3
val_loss= 0.3258488385415659
________________________________
f1_val_H=0.876
epoch= 3
val_loss= 0.36448123057683307
________________________________
f1_val_L=0.799
epoch= 3
val_loss= 0.27629298234687133
________________________________
f1_val_N=0.876
_____________________________________
 
epoch= 4
train_running_loss= 0.8107802578064264
epoch= 4
val_loss= 0.49402421888183146
________________________________
f1_val_A=0.740
epoch= 4
val_loss= 0.4259346290068193
________________________________
f1_val_B=0.785
epoch= 4
val_loss= 0.2310361498966813
________________________________
f1_val_C=0.910
epoch= 4
val_loss= 0.3410783359190313
________________________________
f1_val_H=0.877
epoch= 4
val_loss= 0.28622927674741455
________________________________
f1_val_L=0.825
epoch= 4
val_loss= 0.24180314909009373
________________________________
f1_val_N=0.890
_____________________________________
 
epoch=

epoch= 17
train_running_loss= 0.672449564940047
epoch= 17
val_loss= 0.4668618440628052
________________________________
f1_val_A=0.750
epoch= 17
val_loss= 0.3774395774711262
________________________________
f1_val_B=0.812
epoch= 17
val_loss= 0.19491926953196526
________________________________
f1_val_C=0.927
epoch= 17
val_loss= 0.29241288217102607
________________________________
f1_val_H=0.904
epoch= 17
val_loss= 0.2797810150818391
________________________________
f1_val_L=0.843
epoch= 17
val_loss= 0.21271037003573248
________________________________
f1_val_N=0.913
_____________________________________
 
epoch= 18
train_running_loss= 0.6003208880501562
epoch= 18
val_loss= 0.5211962461471558
________________________________
f1_val_A=0.733
epoch= 18
val_loss= 0.37739824706857855
________________________________
f1_val_B=0.820
epoch= 18
val_loss= 0.21058058769752583
________________________________
f1_val_C=0.916
epoch= 18
val_loss= 0.30621554466282447
________________________________
f1

epoch= 10
val_loss= 0.19776429096236825
________________________________
f1_val_C=0.920
epoch= 10
val_loss= 0.3494018010976838
________________________________
f1_val_H=0.876
epoch= 10
val_loss= 0.31836167971293133
________________________________
f1_val_L=0.825
epoch= 10
val_loss= 0.23156231028192184
________________________________
f1_val_N=0.907
_____________________________________
 
epoch= 11
train_running_loss= 0.668467094625817
epoch= 11
val_loss= 0.522473016205956
________________________________
f1_val_A=0.737
epoch= 11
val_loss= 0.4390399076721885
________________________________
f1_val_B=0.802
epoch= 11
val_loss= 0.2646877483154337
________________________________
f1_val_C=0.906
epoch= 11
val_loss= 0.31132322295409876
________________________________
f1_val_H=0.873
epoch= 11
val_loss= 0.26034225117076526
________________________________
f1_val_L=0.846
epoch= 11
val_loss= 0.22687756138689377
________________________________
f1_val_N=0.910
_____________________________________

epoch= 3
val_loss= 0.2588481131721945
________________________________
f1_val_N=0.893
_____________________________________
 
epoch= 4
train_running_loss= 0.798751405300913
epoch= 4
val_loss= 0.4880525908049415
________________________________
f1_val_A=0.765
epoch= 4
val_loss= 0.4470265345139937
________________________________
f1_val_B=0.791
epoch= 4
val_loss= 0.26300390623509884
________________________________
f1_val_C=0.902
epoch= 4
val_loss= 0.34508463740348816
________________________________
f1_val_H=0.875
epoch= 4
val_loss= 0.3332649004278761
________________________________
f1_val_L=0.820
epoch= 4
val_loss= 0.2442408931605956
________________________________
f1_val_N=0.904
_____________________________________
 
epoch= 5
train_running_loss= 0.8154583471864003
epoch= 5
val_loss= 0.5162370187394759
________________________________
f1_val_A=0.732
epoch= 5
val_loss= 0.4933075931939212
________________________________
f1_val_B=0.763
epoch= 5
val_loss= 0.26947710383683443
__________

epoch= 17
val_loss= 0.3776026395234195
________________________________
f1_val_B=0.818
epoch= 17
val_loss= 0.19635441526770592
________________________________
f1_val_C=0.923
epoch= 17
val_loss= 0.3051931821718449
________________________________
f1_val_H=0.897
epoch= 17
val_loss= 0.3014746597318938
________________________________
f1_val_L=0.836
epoch= 17
val_loss= 0.24315063173280044
________________________________
f1_val_N=0.898
_____________________________________
 
epoch= 18
train_running_loss= 0.6485674937569194
epoch= 18
val_loss= 0.4117083111230065
________________________________
f1_val_A=0.772
epoch= 18
val_loss= 0.35511851581660187
________________________________
f1_val_B=0.826
epoch= 18
val_loss= 0.18994878946493068
________________________________
f1_val_C=0.927
epoch= 18
val_loss= 0.29569432321118144
________________________________
f1_val_H=0.899
epoch= 18
val_loss= 0.2915462138074817
________________________________
f1_val_L=0.840
epoch= 18
val_loss= 0.20015694683088

epoch= 10
val_loss= 0.31949428672140295
________________________________
f1_val_L=0.828
epoch= 10
val_loss= 0.23628683212925405
________________________________
f1_val_N=0.906
_____________________________________
 
epoch= 11
train_running_loss= 0.6793905324199495
epoch= 11
val_loss= 0.5533643172067755
________________________________
f1_val_A=0.743
epoch= 11
val_loss= 0.4396169564940713
________________________________
f1_val_B=0.791
epoch= 11
val_loss= 0.2366775581613183
________________________________
f1_val_C=0.910
epoch= 11
val_loss= 0.3182623379114198
________________________________
f1_val_H=0.884
epoch= 11
val_loss= 0.27773935731613275
________________________________
f1_val_L=0.846
epoch= 11
val_loss= 0.2273088632261052
________________________________
f1_val_N=0.907
_____________________________________
 
epoch= 12
train_running_loss= 0.6849822567928588
epoch= 12
val_loss= 0.46499691815937266
________________________________
f1_val_A=0.758
epoch= 12
val_loss= 0.4300580322742